The architecture of the Neural Network is:
- Input layer with 784 neurons
- Hidden layer with 100 neurons
- Output layer with 10 neurons (number of classes)

Just as the output layer the hidden layer can be considered as a layer with 100 classes. The input to each neuron in the hidden layer is the weighted sum of each feature of a training example. To this input the hidden layer applies the ReLU function. This output goes as input to the output layer of the network where each neuron again receives the input as the weighted sum of ReLU values and second layer weights. 

For one training example the dimensions of the network are:
- Input layer: $X^{(1)}$ => 784 $\times$ 1
- First layer weights: $W^{(1)}$ => 100 $\times$ 784 
- Hidden layer: $Z^{(2)} = W^{(1)} X^{(1)}$ => 100 $\times$ 1
- Hidden layer activated: $a^{(2)} = f(Z^{(2)})$ => 100 $\times$ 1, where $f(x) = max(0, x)$ i.e. ReLU
- Second/Output layer weights: $W^{(2)}$ => 10 $\times$ 100
- Output layer: $Z^{(3)} = W^{(2)} a^{(2)}$ => 10 $\times$ 1
- Output: Softmax classifier, 
\begin{equation}
    \hat{y} = \frac{\mathrm{e}^{Z^{(3)}}}{\sum_{C}\mathrm{e}^{Z^{(3)}}}
\end{equation}

where C is the number of classes (10 in this case)

This network uses the Cross Entropy Loss Function which is of the form:
\begin{equation}
    E = - \sum\limits_{j=1}^{C}t_{j}\log{\hat{y}_{j}}
\end{equation}

For backpropagation we need to calculate the derivative of E w.r.t. $W^{(2)}$ and $W^{(1)}$.
Using the chain rule we get (for one training example):
\begin{equation}
    \frac{\partial{E}}{\partial{W^{(2)}}} = - \frac{\partial{E}}{\partial{\hat{y}_{j}}}
    \frac{\partial\hat{y}_{j}}{\partial{Z^{(3)}}}
    \frac{\partial{Z^{(3)}}}{\partial{W^{(2)}}}
\end{equation}
and
\begin{equation}
    \frac{\partial{E}}{\partial{W^{(1)}}} = - \frac{\partial{E}}{\partial{\hat{y}_{j}}} 
    \frac{\partial{\hat{y}_{j}}}{\partial{Z^{(3)}}}
    \frac{\partial{Z^{(3)}}}{\partial{a^{(2)}}}
    \frac{\partial{a^{(2)}}}{\partial{Z^{(2)}}}
    \frac{\partial{Z^{(2)}}}{\partial{W^{(1)}}}
\end{equation}

where

\begin{equation}
    \frac{\partial{Z^{(3)}}}{\partial{W^{(2)}}} = a^{(2)}
\end{equation}

\begin{equation}
    \frac{\partial{Z^{(3)}}}{\partial{a^{(2)}}} = W^{(2)}
\end{equation}

\begin{equation}
    \frac{\partial{a^{(2)}}}{\partial{Z^{(2)}}} = f'(Z^{(2)})
\end{equation}

\begin{equation}
    \frac{\partial{Z^{(2)}}}{\partial{W^{(1)}}} = Z^{(1)}
\end{equation}

While calculating the above 4 derivatives is pretty simple but finding the values of $\frac{\partial{E}}{\partial{\hat{y}_{i}}}$ and $\frac{\partial{\hat{y}}_{j}}{\partial{Z}^{(3)}}$ is more tricky. The value of the product $\frac{\partial{E}}{\partial{\hat{y}_{i}}} \frac{\partial{\hat{y}}_{j}}{\partial{Z}^{(3)}}$ is ($\hat{y} - t$) where $\hat{y}$ is the predicted class probability vector and $t$ is the actual one hot encoded vector.

Using the above information we get:

\begin{equation}
    \frac{\partial{E}}{\partial{W}^{(2)}} = - \ (\hat{y} - t)\ a^{(2)}
\end{equation}

and

\begin{equation}
    \frac{\partial{E}}{\partial{W}^{(1)}} = - \ (\hat{y} - t)\  W^{(2)} \ f'(Z^{(2)}) \ Z^{(1)}
\end{equation}

Another inportant thing to note here is that $f(Z^{(2)})$ is the ReLU activation unit and its derivative $f'(Z^{(2)})$ is just a matirx of zeros and ones. So for the derivative the hidden layer matrix ($100 \times 1$) will have ones where the value is greater than 0 and zeros elsewhere.

The below code implements the above neural network architecture.

Import required libraries

In [4]:
import pandas as pd
import numpy as np
import os
import struct
from matplotlib import pyplot
from sklearn.model_selection import train_test_split

Read data into a dataframe

In [ ]:
def read_mnist_to_df(file_name, is_label):
    with open(file_name, 'rb') as mnist_file:
        if is_label:
            magic_nr, size = struct.unpack(">II", mnist_file.read(8))
            data = np.fromfile(mnist_file, dtype=np.uint8)
            pd_data = pd.DataFrame(data, columns = ['digit'])
        else:
            magic_nr, size, rows, cols = struct.unpack(">IIII", mnist_file.read(16))
            data = np.fromfile(mnist_file, dtype=np.uint8).reshape((size, rows*cols)).astype(np.float32)/256.
            pd_data = pd.DataFrame(data, columns = [str(i) for i in range(rows*cols)])
    return pd_data

Create train and test data

In [ ]:
train_pixels = read_mnist_to_df('train-images-idx3-ubyte', False)
train_labels = read_mnist_to_df('train-labels-idx1-ubyte', True)
test_pixels = read_mnist_to_df('t10k-images-idx3-ubyte', False)
test_labels = read_mnist_to_df('t10k-labels-idx1-ubyte', True)

Weight initialization for hidden layer with 100 neurons

In [ ]:
input_layer = train_pixels.shape[1]
hidden_neurons = 100
num_classes = len(set(train_labels.digit))
learn_rate = 0.01

W1 = np.random.randn(hidden_neurons, input_layer) * ((2/input_layer)**(1/2))
b1 = np.zeros((hidden_neurons, 1))
W2 = np.random.randn(num_classes, hidden_neurons)
b2 = np.zeros((num_classes, 1))

xtrain = train_pixels.as_matrix()
xtrain -= np.mean(train_pixels, axis=0)
ytrain = np.zeros((train_labels.shape[0], 10))
for i in range(len(ytrain)):
    ytrain[i, train_labels.digit.iloc[i]] = 1

ytrain = ytrain.astype('int')
    
xtest = test_pixels.as_matrix()
xtest -= np.mean(test_pixels, axis=0)
ytest = np.zeros((test_labels.shape[0], 10))
for i in range(len(ytest)):
    ytest[i, test_labels.digit.iloc[i]] = 1

ytest = ytest.astype('int')

Train the network and predict on test data

In [ ]:
epochTrainLoss = []
epochTestLoss = []
trainActual = train_labels.digit.values
testActual = test_labels.digit.values
regLambda = .01

for j in range(0, 80):

    totalTrainLoss = 0
    delta2 = np.zeros((num_classes, hidden_neurons))
    delta1 = np.zeros((hidden_neurons, input_layer))

    for i in range(0, xtrain.shape[0]):
        #Forward Pass
        X = xtrain[i:i+1, :]
        hidden_layer = np.maximum(0, np.add(np.dot(W1, X.T), b1))
        output_layer = np.add(np.dot(W2, hidden_layer), b2)
        pred = np.exp(output_layer) / np.sum(np.exp(output_layer), axis=0)

        #Loss
        Y = ytrain[i:i+1, :]
        trainLoss = -1 * np.sum(np.multiply(Y.T, np.log(pred)))

        #Backward Pass
        dEdW2 = -1 * np.dot((pred - Y.T), hidden_layer.T)
        temp1 = -1 * np.dot(W2.T, (pred - Y.T))
        temp1[temp1 <= 0] = 0
        dEdW1 = np.dot(temp1, X)

        totalTrainLoss += trainLoss
        delta2 += dEdW2 + (regLambda * W2)
        delta1 += dEdW1 + (regLambda * W1)

    #Weight update
    W2 += learn_rate * (delta2 / xtrain.shape[0])
    W1 += learn_rate * (delta1 / xtrain.shape[0])
    
    hlTrain = np.maximum(0, np.add(np.dot(W1, xtrain.T), b1))
    olTrain = np.add(np.dot(W2, hlTrain), b2)
    finalTrain = np.exp(olTrain) / np.sum(np.exp(olTrain), axis=0)
    trainPreds = np.argmax(finalTrain, axis=0)
    trainAcc = np.sum(trainPreds == trainActual)
    trainAcc = (trainAcc / xtrain.shape[0]) * 100
    
    hlTest = np.maximum(0, np.add(np.dot(W1, xtest.T), b1))
    olTest = np.add(np.dot(W2, hlTest), b2)
    finalTest = np.exp(olTest) / np.sum(np.exp(olTest), axis=0)
    testPreds = np.argmax(finalTest, axis=0)
    testAcc = np.sum(testPreds == testActual)
    testAcc = (testAcc / xtest.shape[0]) * 100
    
    totalTestLoss = (-1 * np.sum(np.sum(np.multiply(ytest.T, np.log(finalTest)), axis=0))) / xtest.shape[0]
    epochTestLoss.append(totalTestLoss)
    epochTrainLoss.append(totalTrainLoss)
    print('Epoch -', (j+1), ':')
    print('Train Loss:', (totalTrainLoss / xtrain.shape[0]), ', Train Accuracy:', trainAcc, 
          ', Test Loss:', totalTestLoss, ', Test Accuracy:', testAcc)